In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras

seed = 42
data_import = pd.read_csv("keypoints.csv", index_col=0)
data_import.head()

,0,0_1,1,1_1,2,2_1,3,3_1,4,4_1,...,16_1,17,17_1,18,18_1,19,19_1,label,20,20_1
0,0.457428,0.583321,0.570786,0.503428,0.640131,0.373976,0.651734,0.264575,0.634491,0.179894,...,0.472125,0.325685,0.335119,0.391186,0.321072,0.404947,0.404261,A,0.403822,0.457566
1,0.491985,0.613174,0.606477,0.536362,0.676951,0.406153,0.690357,0.291870,0.674907,0.209456,...,0.506045,0.358678,0.350373,0.437198,0.361405,0.449585,0.447797,A,0.444951,0.501605
2,0.583805,0.687606,0.708179,0.634471,0.776888,0.506354,0.787617,0.383429,0.812475,0.285312,...,0.671530,0.423932,0.480300,0.436632,0.482285,0.456371,0.586887,A,0.476671,0.648893
3,0.723598,0.674288,0.796444,0.618316,0.847458,0.513981,0.862209,0.426482,0.858674,0.356800,...,0.567408,0.628164,0.489217,0.623643,0.453718,0.630732,0.510247,A,0.640717,0.556273
4,0.695232,0.710189,0.775316,0.622683,0.827593,0.521930,0.842571,0.429177,0.849023,0.354402,...,0.573719,0.611416,0.496050,0.611099,0.452895,0.624936,0.509147,A,0.641204,0.559075


In [3]:
X_data = data_import.drop("label", axis=1)
y_data_raw = data_import['label']

In [8]:
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()
y_data = label_encoder.fit_transform(y_data_raw)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=seed)

In [14]:
num_classes = data_import.label.nunique()
input_shape = X_data.shape[1]

In [16]:
model = keras.models.Sequential([
    keras.layers.Input((input_shape)),
    keras.layers.Dense(50, activation ='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(40, activation = 'relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(30, activation = 'relu'),
    keras.layers.Dense(num_classes, activation = 'softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                2150      
                                                                 
 dropout (Dropout)           (None, 50)                0         
                                                                 
 dense_1 (Dense)             (None, 40)                2040      
                                                                 
 dropout_1 (Dropout)         (None, 40)                0         
                                                                 
 dense_2 (Dense)             (None, 30)                1230      
                                                                 
 dense_3 (Dense)             (None, 28)                868       
                                                                 
Total params: 6,288
Trainable params: 6,288
Non-trainabl

In [17]:
#Callbacks
checkpoint_cb = keras.callbacks.ModelCheckpoint("keypoint_classifier.h5", save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(monitor="val_loss" , patience=10)

# Model Compilation
model.compile(optimizer='adam', 
                loss='sparse_categorical_crossentropy',
                metrics = ['accuracy'])

In [18]:
# Model training

model.fit(X_train, y_train,
            batch_size = 64,
            epochs=500,
            validation_data = (X_test, y_test),
            callbacks = [checkpoint_cb, early_stopping_cb])

Epoch 1/500
19/19 [==============================] - 3s 55ms/step - loss: 3.3352 - accuracy: 0.0482 - val_loss: 3.3089 - val_accuracy: 0.0309
Epoch 2/500
19/19 [==============================] - 0s 16ms/step - loss: 3.3041 - accuracy: 0.0516 - val_loss: 3.2819 - val_accuracy: 0.0893
Epoch 3/500
19/19 [==============================] - 0s 23ms/step - loss: 3.2781 - accuracy: 0.0620 - val_loss: 3.2333 - val_accuracy: 0.1271
Epoch 4/500
19/19 [==============================] - 0s 12ms/step - loss: 3.2265 - accuracy: 0.0947 - val_loss: 3.1651 - val_accuracy: 0.1512
Epoch 5/500
19/19 [==============================] - 0s 14ms/step - loss: 3.1646 - accuracy: 0.1170 - val_loss: 3.0599 - val_accuracy: 0.1375
Epoch 6/500
19/19 [==============================] - 0s 10ms/step - loss: 3.0853 - accuracy: 0.1403 - val_loss: 2.9379 - val_accuracy: 0.1512
Epoch 7/500
19/19 [==============================] - 0s 11ms/step - loss: 2.9645 - accuracy: 0.1463 - val_loss: 2.7675 - val_accuracy: 0.1753
Epoch 

In [19]:
# loading saved model
model = keras.models.load_model("keypoint_classifier.h5")

# Evaluate
model.evaluate(X_test, y_test)

10/10 [==============================] - 0s 4ms/step - loss: 0.3892 - accuracy: 0.9519


[0.38921529054641724, 0.9518900513648987]

In [43]:
# Prediction on random sample

import random
i = random.randint(0, len(X_data))
X = X_data.iloc[i]
y = y_data[i]
pred = model.predict(np.array([X]))
print("True Label:", y)
print("prediction:", np.argmax(pred))

1/1 [==============================] - 0s 32ms/step
True Label: 16
ptrdivtion: 16
